In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
from glob import glob
from tqdm import tqdm
from pathlib import Path
import re

%matplotlib widget
import matplotlib as mp
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm, Normalize, LogNorm
from matplotlib.patches import ConnectionPatch
import textwrap

import functions_calcs as fc
import functions_plotting as fp

SAVE_PLOTS = True

proj_path = Path('/work/pi_kandread_umass_edu/Cloud_Freq/')
figure_path = proj_path / "figures" 

parquet_file = proj_path / 'data' / 'grades_level2.parquet'
site_file = proj_path / 'data' / 'MERIT_sites' / 'merit_stratified_discharge_level2_sample.shp'

hydroBasins_files = glob('/work/pi_kandread_umass_edu/Datasets/HydroBASINS/*/*lev02*.shp')

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = world.to_crs('World_Robinson')

daily_data_file = proj_path / 'data' / 'analysis_data_daily.parquet'
sites_data_file = proj_path / 'data' / 'analysis_data_sites.parquet'
basins_data_file = proj_path / 'data' / 'analysis_data_basins.parquet'
resampled_sites_data_file = proj_path / 'data' / 'analysis_data_resampled_sites.pkl'

/tmp/ipykernel_2765189/2723989440.py:30: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [ ]:
import importlib
importlib.reload(fc)
importlib.reload(fp)

In [ ]:
# read in pre-processed daily data
df = pd.read_parquet(parquet_file, engine='pyarrow')

# Calculate 30-day rolling temp and use Xiao's ice mode
temp30 = (df.groupby('id')['temperature_2m']
          .rolling(window=30, min_periods=1)
          .mean()
          .reset_index(level=0, drop=True))-273.15
period = (df.index.get_level_values('time').month.isin([8, 9, 10, 11, 12, 1])).astype(int)
log_ice = (-0.32*temp30) + (-0.05*temp30*period) + -0.82
df['pIce'] = np.exp(log_ice)

df[df.pIce>0.5] = np.nan
df[df.Q<=0] = np.nan
df.dropna(subset=['Q','cloudMask'], inplace=True)

#Remove basins with mean Q < 1
ids_to_remove = df.groupby('id')['Q'].mean()[lambda x: x < 1].index
df= df[~df.index.get_level_values('id').isin(ids_to_remove)]

# read in site data
sites = gpd.read_file(site_file).set_index('id')
# median q and cloud values per site
sites = sites.join(df.groupby('id').median()).to_crs('World_Robinson')

# Define cloud classes
bins = [0, 0.25, 0.75, 1]
labels = ['No Cloud', 'Mixed', 'Cloud']
# Use pd.cut to map values to bins
df['cloud_class'] = pd.cut(df['cloudMask'], bins=bins, labels=labels, right=False)
df['cloud_binary'] = df['cloud_class'] != 'No Cloud'

# Rank the 'Q' within each site
df['Q_norm'] = fc.calc_quantiles(df)

# Create bins from normalized flow data
num_bins = 10
bin_edges = np.linspace(0, 1, num_bins+1)
df['quantile_bin'] = np.digitize(df['Q_norm'], bins=bin_edges, right=True)

df['month'] = df.index.get_level_values('time').month

# Calculate normalized differences
quants = [0, 1, 5, 10, 50, 90, 95, 99, 100]
for q in quants:
    sites = fc.calc_masked_norm_diff(sites, df['Q'], df['cloud_binary'], q/100, f'normDiff_q{q:02.0f}')

sites = sites.join(fc.calc_spectral_props(df))

#Read in hydrobasins
gdf_list = []
for file in hydroBasins_files:
    gdf_list.append(gpd.read_file(file))
hydroBasins = pd.concat(gdf_list, ignore_index=True).set_index('PFAF_ID')

#clipping removes antimeridian crossovers in reprojection
hydroBasins = gpd.clip(hydroBasins, [-180, -90, 180, 90]).to_crs('World_Robinson')

#Merge level2 average values to hydrobasins
sites['l1'] = sites.index//1E7
sites['l2'] = sites.index//1E6
level2_values = sites.groupby('l2').median(numeric_only=True)
hydroBasins = hydroBasins.merge(level2_values, left_index=True, right_index=True)

# # Calculate seasonal amplitude and phase
# monthly = df.groupby(['id','month']).agg({
#     'Q_norm': ['count','mean'],   
#     'cloudMask': 'mean'
# })
# monthly.columns = ['count', 'Q_norm', 'cloudMask']
# sites['amp'] = np.nan
# sites['offset'] = np.nan
# for idx, g in monthly.groupby('id'):
#     mask = g['count'] > (g['count'].max() * 0.1)
#     tmp = g[mask]   
#     if len(tmp) != 0:
#         phase_offset, amp = fc.calc_phase_amp(tmp['cloudMask'],tmp['Q_norm'])    
#     if ~np.isnan(amp):    
#         sites.loc[idx,'amp'] = amp
#         sites.loc[idx,'offset'] = phase_offset

#Read in the Krabbenhoft 2020 network bias data
tmp = pd.read_csv(proj_path / 'data' / "Krabbenhoft" / "Attribute_table.csv",low_memory=False)
tmp = tmp.set_index('COMID')['stationid']
sites = sites.join(tmp)

quants = [0, 0.01, 0.05, 0.10, 0.50, 0.90, 0.95, 0.99, 1.00]
windows = ['1D','3D','7D','14D','1ME','2ME','3ME','6ME','1YE']
resampled_sites = fc.resampled_norm_diff(sites,df,windows,quants)

df.to_parquet(daily_data_file, engine='pyarrow')
sites.to_parquet(sites_data_file, engine='pyarrow')
hydroBasins.to_parquet(basins_data_file, engine='pyarrow')
resampled_sites.to_pickle(resampled_sites_data_file)

print("Done!")

In [2]:
df = pd.read_parquet(daily_data_file, engine='pyarrow')
sites = gpd.read_parquet(sites_data_file)
hydroBasins = gpd.read_parquet(basins_data_file)

In [3]:
df

,,cloudMask,temperature_2m,Q,pIce,cloud_class,cloud_binary,Q_rank,Q_norm,quantile_bin,month
time,id,,,,,,,,,,
2000-02-26,11004678,0.000000,305.050198,0.019567,0.000016,No Cloud,False,2094.0,0.248841,3,2
2000-02-27,11004678,0.000000,303.389604,0.019322,0.000021,No Cloud,False,2074.0,0.246463,3,2
2000-02-28,11004678,0.003130,301.492652,0.020493,0.000028,No Cloud,False,2174.0,0.258352,3,2
2000-03-01,11004678,0.040691,301.278195,0.021155,0.000033,No Cloud,False,2242.0,0.266437,3,3
2000-03-02,11004678,0.679383,300.944331,0.020203,0.000038,Mixed,True,2150.0,0.255499,3,3
...,...,...,...,...,...,...,...,...,...,...,...
2023-08-27,91006323,0.069345,272.328320,15.021329,0.209224,No Cloud,False,208.0,0.192917,2,8
2023-08-28,91006323,0.005727,271.535291,15.046756,0.224514,No Cloud,False,217.0,0.201305,3,8
2023-08-29,91006323,0.103834,269.996463,15.059526,0.240861,No Cloud,False,221.0,0.205033,3,8


In [ ]:
fig, ax = plt.subplots(figsize=(4,3))
fp.cloud_bar_plot(df, ax)
plt.tight_layout()

if SAVE_PLOTS:
    fig.savefig(figure_path / "cloudiness_all.png", format='png', dpi=600)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

quant = 50
norm = TwoSlopeNorm(vmin=-0.4, vcenter=0, vmax=0.4)
cmap = 'RdBu'

fp.pfaf_level2_plot(hydroBasins,f'normDiff_q{quant:02.0f}',cmap,norm,ax)

plt.title(f"Normalized Difference at {quant:02.0f}% Quantile")

fig.colorbar(mp.cm.ScalarMappable(norm=norm, cmap=cmap), 
             ax=ax, 
             fraction=0.05, 
             shrink=0.7, 
             pad=0.05,
             label='')


In [ ]:
plt.close('all')

norm = TwoSlopeNorm(vmin=-0.5, vcenter=0, vmax=0.5)
cmap = 'RdBu'
    
fig, axes = plt.subplots(3,3,figsize=(10,5))
axes = axes.flatten()

for ax, quant in tqdm(zip(axes,quants),total=len(axes)):
    fp.pfaf_level2_plot(hydroBasins,f'normDiff_q{quant:02.0f}',cmap, norm, ax)
    ax.set_title(f"{quant:2.0f}%",y=0.9)
    ax.axis('off')

plt.subplots_adjust(left=0.05, right=0.85, top=0.95, bottom=0.05, wspace=0, hspace=0)
cax = fig.add_axes([0.87, 0.2, 0.02, 0.6])
cbar = fig.colorbar(mp.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax)
cbar.set_ticks([-0.4, -0.2, 0, 0.2, 0.4])
cbar.set_label("Normalized difference")

if SAVE_PLOTS:
    fig.savefig(figure_path / "3x3_maps.png", format='png', dpi=600)

In [ ]:
# weird_id = 2260400 #IRRAWADDY RIVER
weird_site = sites.sample(1)
weird_id = weird_site.index[0]

weird_df = df[df.index.get_level_values('id')==weird_id]

plt.close('all')
fig, axes = plt.subplot_mosaic([['top','top'],['left', 'right']],
                              constrained_layout=True)


axes['top'].plot(weird_df.index.get_level_values('time'),
         weird_df['Q'])
axes['top'].scatter(weird_df.index.get_level_values('time')[~weird_df.cloud_binary],
            weird_df['Q'][~weird_df.cloud_binary],color='orange')

fp.cloud_bar_plot(weird_df, axes['left'])

world.plot(ax=axes['right'], color='lightgray')
weird_site.to_crs('World_Robinson').plot(ax=axes['right'])
# axes['right'].square()
axes['right'].set_xlim([-1.4E7, 1.6E7])
axes['right'].set_ylim([-6E6, 8E6])
axes['right'].axis('off')

In [ ]:
plt.close('all')
norm = TwoSlopeNorm(vmin=-0.5, vcenter=0, vmax=0.5)
cmap = 'RdBu'

fig, ax = plt.subplots(figsize=(5,4))
ax.scatter(sites['amp'],
            sites['offset']+np.random.rand(len(sites))*0,
            c=sites['normDiff_q50'],s=0.1,norm=norm, cmap='RdBu',edgecolor=None)
ax.set_ylabel("Phase offset")
ax.set_xlabel("Mean amplitude")
ax.set_xlim([0.05,0.5])
# ax.set_facecolor([0.75]*3)

plt.subplots_adjust(left=0.1, right=0.8)
cax = fig.add_axes([0.85, 0.15, 0.03, 0.7])
cbar = fig.colorbar(mp.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax)
cbar.set_ticks([-0.4, -0.2, 0, 0.2, 0.4])
cbar.set_label("Norm. diff. of median discharge")

if SAVE_PLOTS:
    fig.savefig(figure_path / "phase_amplitude_scatter.png", format='png', dpi=600)

In [ ]:
plt.close('all')
col_plot = 'uparea'
river_sizes = [0,1E2,1E3,1E4,1E5,1E6,np.inf] #basin size

# Create subplots
fig, ax = plt.subplots(figsize=(4,3))

# Create a color cycle with a continuous colormap
colors = plt.cm.viridis(np.linspace(0, 1, len(river_sizes)-1))
ax.set_prop_cycle('color', colors)

# Plot each site ID's time series
for lower_limit, upper_limit in zip(river_sizes, river_sizes[1:]):
    # sites_size_bin = sites[(sites['lta_discharge']>=lower_limit) & (sites['lta_discharge']<upper_limit)]
    sites_size_bin = sites[(sites[col_plot]>=lower_limit) & (sites[col_plot]<upper_limit)]
    df_size_bin = df[df.index.get_level_values('id').isin(sites_size_bin.index)]

    df_grouped = df_size_bin.groupby('quantile_bin')['cloud_class'].value_counts(normalize=True).unstack('cloud_class')
    df_grouped = df_grouped[(df_grouped.index > 0) & (df_grouped.index<11)]


    ax.plot(df_grouped.index,df_grouped['No Cloud'],label=f"{lower_limit/1E3:.0f}-{upper_limit/1E3:.0f}, n={len(sites_size_bin)}")

ax.set_xticks(np.linspace(1,10,10))
new_labels = [f'{label * 0.1:.1f}' for label in df_grouped.index]
ax.set_xticklabels(new_labels, rotation=0)

# Create a twin Axes to duplicate y labels
ax2 = ax.twinx()
ax2.set_yticks(ax.get_yticks())
ax2.set_ylim(ax.get_ylim())

# Adjust layout and show the plots
ax.legend(title='Area (1000 km\u00B2)')
ax.set_xlabel('Discharge Quantile')
ax.set_ylabel('Fraction of cloud-free images')
plt.tight_layout()
plt.show()

if SAVE_PLOTS:
    fig.savefig(figure_path / "cloudiness_by_area.png", format='png', dpi=600)

In [ ]:
plt.close('all')

col_plot = 'Q'
river_sizes = [1E0, 1E1, 1E2, 1E3, 1E4, 1E6]

# Create subplots
fig, ax = plt.subplots(figsize=(6,3))

# Create a color cycle with a continuous colormap
colors = plt.cm.viridis(np.linspace(0, 1, len(river_sizes)-1))
ax.set_prop_cycle('color', colors)

# Plot each site ID's time series
for lower_limit, upper_limit in zip(river_sizes, river_sizes[1:]):
    # sites_size_bin = sites[(sites['lta_discharge']>=lower_limit) & (sites['lta_discharge']<upper_limit)]
    sites_size_bin = sites[(sites[col_plot]>=lower_limit) & (sites[col_plot]<upper_limit)]
    df_size_bin = df[df.index.get_level_values('id').isin(sites_size_bin.index)]

    df_grouped = df_size_bin.groupby('quantile_bin')['cloud_class'].value_counts(normalize=True).unstack('cloud_class')
    df_grouped = df_grouped[(df_grouped.index > 0) & (df_grouped.index < 11)]

    low_string = f'$10^{np.log10(lower_limit):1.0f}$'
    high_string = f'$10^{np.log10(upper_limit):1.0f}$'
    
    ax.plot(df_grouped.index,df_grouped['No Cloud'],label=f"{low_string}-{high_string}, n={len(sites_size_bin)}")
    print(df_grouped['No Cloud'])

ax.set_xticks(np.linspace(2,10,5))
new_labels = [f'{label * 0.1:.1f}' for label in ax.get_xticks()]
ax.set_xticklabels(new_labels, rotation=0)

ax.set_ylim([0.23,0.6])
ax.set_yticks(np.linspace(0.25,0.55,4))

# # Create a twin Axes to duplicate y labels
# ax2 = ax.twinx()
# ax2.set_yticks(ax.get_yticks())
# ax2.set_ylim(ax.get_ylim())

# Adjust layout and show the plots
plt.tight_layout()
plt.subplots_adjust(left=0.1, right=0.5, bottom=0.2)
ax.legend(title='Discharge [$m^3/s$]', loc='upper left', bbox_to_anchor=(1.2, 1))
ax.set_xlabel('Discharge Quantile')
ax.set_ylabel('Fraction of cloud-free images')

plt.show()

if SAVE_PLOTS:
    fig.savefig(figure_path / "cloudiness_by_discharge.png", format='png', dpi=600)

In [ ]:
plt.close('all')
tmp.correlation.hist(range=(-.6,.6),bins=11)

In [ ]:
plt.close('all')

col_plot = 'temperature_2m'
river_sizes = [0,10,15,20,25,30]

# Create subplots
fig, ax = plt.subplots(figsize=(6,3))

# Create a color cycle with a continuous colormap
colors = plt.cm.viridis(np.linspace(0, 1, len(river_sizes)-1))
ax.set_prop_cycle('color', colors)

# Plot each site ID's time series
for lower_limit, upper_limit in zip(river_sizes, river_sizes[1:]):
    sites_size_bin = sites[(sites[col_plot]>=(lower_limit+273.15)) & (sites[col_plot]<(upper_limit+273.15))]
    df_size_bin = df[df.index.get_level_values('id').isin(sites_size_bin.index)]

    df_grouped = df_size_bin.groupby('quantile_bin')['cloud_class'].value_counts(normalize=True).unstack('cloud_class')
    df_grouped = df_grouped[(df_grouped.index > 0) & (df_grouped.index < 11)]

    low_string = f'{lower_limit}'
    high_string = f'{upper_limit}'
    
    ax.plot(df_grouped.index,df_grouped['No Cloud'],label=f"{low_string} - {high_string}, n={len(sites_size_bin)}")

ax.set_xticks(np.linspace(2,10,5))
new_labels = [f'{label * 0.1:.1f}' for label in ax.get_xticks()]
ax.set_xticklabels(new_labels, rotation=0)

ax.set_ylim([0.23,0.6])
ax.set_yticks(np.linspace(0.25,0.55,4))

# # Create a twin Axes to duplicate y labels
# ax2 = ax.twinx()
# ax2.set_yticks(ax.get_yticks())
# ax2.set_ylim(ax.get_ylim())

# Adjust layout and show the plots
plt.tight_layout()
plt.subplots_adjust(left=0.1, right=0.5, bottom=0.2)
ax.legend(title='Temperature [C]', loc='upper left', bbox_to_anchor=(1.2, 1))
ax.set_xlabel('Discharge Quantile')
ax.set_ylabel('Fraction of cloud-free images')

plt.show()

if SAVE_PLOTS:
    fig.savefig(figure_path / "cloudiness_by_temperature.png", format='png', dpi=600)

In [ ]:
plt.close('all')

col_plot = 'slope'
river_sizes = [1E-6, 1E-4, 1E-3, 1E-2, 1E0]

# Create subplots
fig, ax = plt.subplots(figsize=(6,3))

# Create a color cycle with a continuous colormap
colors = plt.cm.viridis(np.linspace(0, 1, len(river_sizes)-1))
ax.set_prop_cycle('color', colors)

# Plot each site ID's time series
for lower_limit, upper_limit in zip(river_sizes, river_sizes[1:]):
    # sites_size_bin = sites[(sites['lta_discharge']>=lower_limit) & (sites['lta_discharge']<upper_limit)]
    sites_size_bin = sites[(sites[col_plot]>=lower_limit) & (sites[col_plot]<upper_limit)]
    df_size_bin = df[df.index.get_level_values('id').isin(sites_size_bin.index)]

    df_grouped = df_size_bin.groupby('quantile_bin')['cloud_class'].value_counts(normalize=True).unstack('cloud_class')
    df_grouped = df_grouped[(df_grouped.index > 0) & (df_grouped.index < 11)]

    low_string = f'$10^{{ {np.log10(lower_limit):1.0f} }}$'
    high_string = f'$10^{{ {np.log10(upper_limit):1.0f} }}$'
    
    ax.plot(df_grouped.index,df_grouped['No Cloud'],label=f"{low_string}-{high_string}, n={len(sites_size_bin)}")

ax.set_xticks(np.linspace(2,10,5))
new_labels = [f'{label * 0.1:.1f}' for label in ax.get_xticks()]
ax.set_xticklabels(new_labels, rotation=0)

ax.set_ylim([0.23,0.6])
ax.set_yticks(np.linspace(0.25,0.55,4))


# Adjust layout and show the plots
plt.tight_layout()
plt.subplots_adjust(left=0.1, right=0.5, bottom=0.2)
ax.legend(title='Slope [1/m]', loc='upper left', bbox_to_anchor=(1.2, 1))
ax.set_xlabel('Discharge Quantile')
ax.set_ylabel('Fraction of cloud-free images')

plt.show()

if SAVE_PLOTS:
    fig.savefig(figure_path / "cloudiness_by_slope.png", format='png', dpi=600)

In [ ]:
idx = []
corr = []
for i,g in tqdm(df.groupby('id')):
    g.dropna(subset=['cloudMask','Q'], inplace=True)
    
    idx.append(i)
    corr.append(np.corrcoef(g['cloudMask'],g['Q'])[0,1])

In [ ]:
tmp = pd.DataFrame({'correlation': corr},index=idx)
tmp = sites.join(tmp)
tmp['log_slope'] = np.log10(tmp.slope)
tmp['log_Q'] = np.log10(tmp.Q)
tmp.dropna(subset=['slope','Q','temperature_2m','correlation'],inplace=True)

plt.close('all')
tmp.plot.scatter('temperature_2m','correlation',c='normDiff_q50',s=0.25,cmap=cmap,norm=norm)

In [ ]:
quants = [0, 1, 5, 10, 50, 90, 95, 99, 100]
fig, axes = plt.subplots(3,3,figsize=(10,10))
axes = axes.flatten() # Flatten the axes array for easy iteration

sites_plot = sites

for (quant, ax) in tqdm(zip(quants, axes), total=len(quants)): 
    fp.obs_true_1to1(df,quant,ax)
    
fig.supxlabel('All Discharge [$m^3/s$]', fontsize=14)
fig.supylabel('Cloud-Free Discharge [$m^3/s$]', fontsize=14)
plt.tight_layout()

if SAVE_PLOTS:
    fig.savefig(figure_path / "3x3_scatterplots.png", format='png', dpi=600)


In [ ]:
# Function to find length of cloud-covered periods
def cloud_covered_periods(data):
    periods = []
    current_period = 0

    for value in data:
        if value:  # Cloud covered
            current_period += 1
        else:  # Not cloud covered
            periods.append(current_period)
            current_period = 0

    return np.quantile(periods,[q/100.0 for q in quants])
    # return np.mean(periods)

# Apply the function to each 'cloud_binary' series
cloud_periods = df.groupby('id')['cloud_binary'].apply(cloud_covered_periods)

tmp = pd.DataFrame(cloud_periods.tolist(), index=cloud_periods.index)
tmp.columns = [f"q{q:02.0f}" for q in quants]


In [ ]:
fig,ax = plt.subplots(figsize=(10,5))

norm = Normalize(vmin=0, vmax=30)
cmap = 'inferno'

sites_tmp = sites.join(tmp)
sites_tmp.plot('q90',markersize=1,ax=ax,cmap=cmap,norm=norm)

plt.subplots_adjust(left=0.05, right=0.85, top=0.95, bottom=0.05, wspace=0, hspace=0)
cax = fig.add_axes([0.87, 0.2, 0.02, 0.6])
cbar = fig.colorbar(mp.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax)




In [ ]:
from scipy.signal import welch, coherence, csd
plt.close('all')


def spectral_analysis_site(df, site_id):
    # Filter data for the specific site
    site_data = df.xs(site_id, level='id')[['Q_norm','cloudMask']]
    
    #number of valid days per year. Excludes missing data or ice flagged days.
    nperseg = 365.25*len(site_data)/(site_data.index.max()-site_data.index.min()).days

    # Preprocessing: Interpolate missing values and ensure uniform sampling
    site_data = site_data#.resample('D').interpolate()

    # Extract discharge (Q) and cloud cover (cloudMask) columns
    discharge = site_data['Q_norm'].values
    cloud_cover = site_data['cloudMask'].values
    
    # discharge = (discharge-np.mean(discharge))/np.std(discharge)

    # Spectral analysis
    fs = 1  # Sampling frequency (1 sample per day)
    f_discharge, Pxx_discharge = welch(discharge, fs=fs, nperseg=nperseg, scaling='density')
    f_cloud, Pxx_cloud = welch(cloud_cover, fs=fs, nperseg=nperseg, scaling='density')
    f_coherence, Cxy = coherence(discharge, cloud_cover, fs=fs, nperseg=nperseg)

    # Plotting
    fig, ax = plt.subplot_mosaic([['ts','ts','ts'],
                                   ['l','c','r']],
                                   figsize = (8,4))

    # Discharge and cloud cover time series
    ax['ts'].plot(site_data.index, discharge, label='Discharge')
    # ax['ts'].plot(site_data.index, cloud_cover, label='Cloud Cover')
    # ax['r'].xlabel('Time')
    # plt.ylabel('Value')
    # plt.title(f'Time Series (Site {site_id})')
    # plt.legend()

    # Spectral power density
    ax['l'].semilogy(f_discharge, Pxx_discharge, label='Discharge')
    ax['l'].semilogy(f_cloud, Pxx_cloud, label='Cloud Cover')
    # plt.xlabel('Frequency (cycles/day)')
    # plt.ylabel('Power Density')
    # plt.title('Spectral Power Density')
    # plt.legend()

    # Coherence
    ax['c'].plot(f_coherence, Cxy)
    # ax['r'].xlabel('Frequency (cycles/day)')
    # ax['r'].set_ylabel('Coherence')
    # ax['r'].title('Coherence between Discharge and Cloud Cover')
    
    # Calculate cross spectral density which contains phase information
    f, Pxy = csd(discharge, cloud_cover, fs=fs, nperseg=nperseg)

    # Calculate the phase spectrum in degrees
    phase_spectrum = np.angle(Pxy, deg=True)

    # Plot the phase spectrum
    ax['r'].plot(f, phase_spectrum,'.')  # Plotting against period (1/frequency)
    # ax['r'].xlabel('Period (days)')
    # plt.ylabel('Phase Difference (degrees)')
    # plt.title('Phase Spectrum between Discharge and Cloud Cover')
    ax['r'].set_ylim(-180, 180)  # Phase difference ranges from -180 to 180 degrees
    
    # plt.grid(True)

    plt.tight_layout()
    plt.show()
    
    return f

# Example usage for a specific site
site_2_plot = sites.sample()
f = spectral_analysis_site(df, site_2_plot.index[0])
site_2_plot

In [ ]:
from scipy.signal import welch, coherence, csd

idx = []  
n_groups = len(df.index.get_level_values('id').unique())
for i,g in tqdm(df.groupby('id'),total=n_groups):
        g = g.droplevel('id')

        #number of valid days per year. Excludes missing data or ice flagged days.
        # nperseg = 365.25*len(g)/(g.index.max()-g.index.min()).days
        # if nperseg < 180:
        #     continue
            
        nperseg = 90

        if (len(g) < nperseg):
            continue
        
        # Extract discharge (Q) and cloud cover (cloudMask) columns
        discharge = g['Q'].values
        cloud_cover = g['cloudMask'].values

        # Spectral analysis
        fs = 1  # Sampling frequency (1 sample per day)
        f_discharge, Pxx_discharge = welch(discharge, fs=fs, nperseg=nperseg, scaling='density')
        f_cloud, Pxx_cloud = welch(cloud_cover, fs=fs, nperseg=nperseg, scaling='density')
        f_coherence, Cxy = coherence(discharge, cloud_cover, fs=fs, nperseg=nperseg)

        # Calculate cross spectral density which contains phase information
        f, Pxy = csd(discharge, cloud_cover, fs=fs, nperseg=nperseg)
        
        
        count = len(idx)
        if count==0:
            phase = np.full((Cxy.size,n_groups),np.nan)
            power = np.full((Cxy.size,n_groups),np.nan)
            coh = np.full((Cxy.size,n_groups),np.nan)
        phase[:,count] = np.angle(Pxy)
        power[:,count] = Pxx_discharge
        coh[:,count] = Cxy
        
        idx.append(i)

#Select columns that are not all NaN
phase = phase[:, ~np.all(np.isnan(phase), axis=0)]
coh = coh[:, ~np.all(np.isnan(coh), axis=0)]


# tmp = pd.DataFrame({'CSD_phase':phase,'spec_coh':coh},index=idx)
# sites_plot = sites.loc[:, ~sites.columns.isin(['CSD_phase','spec_coh'])]
# sites_plot = sites_plot.join(tmp)


In [ ]:
phase_mean = np.nanmean(phase, axis=1)
power_mean = np.nanmean(power, axis=1)
coh_mean = np.nanmean(coh, axis=1)

plt.close('all')
fig, ax = plt.subplots()
# Create a pseudocolor plot
cax = ax.pcolor(coh[1:,:], edgecolors='none', linewidths=1, vmin=0, vmax=0.3)


In [ ]:
coh.max()

In [ ]:
plt.close('all')
norm = TwoSlopeNorm(vmin=-0.3, vcenter=0, vmax=0.3)
cmap = 'RdBu'
# thresh = 0
fig, ax = plt.subplots(figsize=(5,4))
ax.scatter(sites_plot['spec_coh'],
            np.abs(sites_plot['CSD_phase']),
            c=sites_plot['normDiff_q90'],s=0.5,norm=norm, cmap='RdBu',edgecolor=None)
ax.set_ylabel("Phase offset")
ax.set_xlabel("Mean amplitude")
ax.set_xlim([0.1,1])
# ax.set_facecolor([0.75]*3)

plt.subplots_adjust(left=0.1, right=0.8)
cax = fig.add_axes([0.85, 0.15, 0.03, 0.7])
cbar = fig.colorbar(mp.cm.ScalarMappable(norm=norm, cmap=cmap), cax=cax)
# cbar.set_ticks([-0.4, -0.2, 0, 0.2, 0.4])
cbar.set_label("Norm. diff. of median discharge")


In [ ]:
# Create the 2D histogram bins
x = sites_plot['spec_coh']
y = np.abs(sites_plot['CSD_phase'])
z = sites_plot['normDiff_q90']

norm = TwoSlopeNorm(vmin=-0.5, vcenter=0, vmax=0.5)
cmap = 'RdBu'

# Create 2D histogram for binning the x and y data
x_edges = np.linspace(0, 1, 9)
y_edges = np.linspace(0, np.pi, 9)
arr_size = (len(x_edges)-1, len(y_edges)-1)

x_center = (x_edges[:-1] + x_edges[1:]) / 2
y_center = (y_edges[:-1] + y_edges[1:]) / 2
xx, yy = np.meshgrid(x_center, y_center, indexing='ij')

# Create an array to store the average z values for each bin
z_avg = np.full(arr_size, 0, dtype=np.float64)
z_count = np.full(arr_size, 0, dtype=np.float64)

# Populate the array with the average z values
for i in range(arr_size[0]):
    for j in range(arr_size[1]):
        mask = (x >= x_edges[i]) & (x < x_edges[i + 1]) & (y >= y_edges[j]) & (y < y_edges[j + 1])
        if np.sum(mask)>0:
            z_avg[i, j] = np.mean(z[mask])
            z_count[i, j] = np.sum(mask)

# Plot the 2D histogram with the average z values
plt.close('all')
fig, ax = plt.subplots(figsize=(5.5, 3.5))

ax.set_xlabel("Coherence")
ax.set_ylabel("Cross Spectral Density Phase")
ax.set_ylim([0,np.pi])
ax.set_yticks(np.pi*np.array([0,1/4,1/2,3/4,1]))
ax.set_yticklabels(['0', r'$\frac{1}{4}\pi$', r'$\frac{1}{2}\pi$', r'$\frac{3}{4}\pi$', r'${\pi}$'])
# ax.set_xticks([0.1,0.2,0.3,0.4,0.5])
# ax.set_xlim([0,20])

RESCALE = 4
scatter = ax.scatter(xx,yy,c=z_avg,s=z_count/RESCALE,cmap=cmap,norm=norm)

handles, labels = scatter.legend_elements("sizes",num=6,alpha=0.5)
#Extract the numeric labels and rescale them
rescaled_labels = [f"{int(re.search(r'\d+', label).group()) * RESCALE:.0f}" for label in labels]
legend2 = ax.legend(handles, rescaled_labels, 
                    loc="upper left", 
                    title="River\nreaches", 
                    labelspacing=1.5,
                    frameon=False,
                    # borderpad=1,
                    bbox_to_anchor=(1, 1))

# Adjust subplot and add colorbar
plt.subplots_adjust(left=0.12, right=0.6, bottom=0.15)
cax = fig.add_axes([0.8, 0.15, 0.03, 0.7])
cbar = fig.colorbar(scatter,cax=cax)
cbar.set_label('ND of median')
cbar.set_ticks([-0.4, -0.2, 0, 0.2, 0.4])

# plt.tight_layout()
plt.show()

In [ ]:
plt.close('all')
plt.hist(sites_plot['spec_power'])

In [ ]:
plt.close('all')
# fig, axes = plt.subplots(1,2,figsize=(8,3))
# axes = axes.flatten() # Flatten the axes array for easy iteration

plt.hist(sites.normDiff_q50, range=(-1, 1), bins=41, density=True)
plt.hist(sites[sites.stationid.notna()].normDiff_q50, range=(-1, 1), bins=41, density=True,alpha=0.5)

In [ ]:
pca

In [ ]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression

norm = TwoSlopeNorm(vmin=-0.5, vcenter=0, vmax=0.5)
cmap = 'RdBu'

X = sites.drop(columns=['strmDrop_t','slope_taud','maxup','up1','up2','up3','up4','l1','l2','NextDownID','lengthdir','CSD_phase','CSD_magnitude','coherence','pIce'])
numeric_columns = X.select_dtypes(include=[np.number]).columns
X = X[numeric_columns]
X = X.dropna()

y = X['normDiff_q50']
X = X[[col for col in X.columns if not col.startswith('normDiff')]]

X_standardized = (X - X.mean()) / X.std()

# Perform PCA
pca = PCA(n_components=8)
X_pca = pca.fit_transform(X_standardized)

# Identify predictive variables using Linear Regression

reg = LinearRegression()
reg.fit(X_standardized, y)
coefficients = pd.Series(reg.coef_, index=pca.feature_names_in_)

# Create plots
plt.figure(figsize=(8, 4))

# Scatter plot of PCA components
plt.subplot(1, 2, 1)
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=y, s=0.1, cmap=cmap, norm=norm)
plt.colorbar()
plt.xlabel('PCA Component 1')
plt.ylabel('PCA Component 2')
plt.title('PCA Components')

# Coefficient plot
plt.subplot(1, 2, 2)
coefficients.plot(kind='barh')
plt.xlabel('Coefficient Value')
plt.title('Coefficients for Predicting normDiff_q50')

plt.tight_layout()
plt.show()

In [ ]:
pca.feature_names_in_

In [ ]:
loadings = pca.components_
strength = pca.explained_variance_

# Set the width of the bars
bar_width = 0.1

# Create a bar chart with grouped bars
plt.figure(figsize=(8, 4))
ax = plt.gca()
n_features = loadings.shape[1]
index = np.arange(n_features)
for idx in range(len(loadings)):
    ax.bar(index+(bar_width*idx), loadings[idx]*strength[idx], bar_width)
plt.xlabel('Features')
plt.ylabel('Loadings')
plt.title('PCA Loadings')
plt.xticks(index + bar_width, pca.feature_names_in_, rotation=90)
# plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
pca.explained_variance_

In [ ]:
# Get the PCA loadings
loadings = pca.components_
n_comp = len(loadings)

# Calculate the overall strength of each axis
strength_axis = pca.explained_variance_ratio_

# Create a bar chart
plt.figure(figsize=(6, 6))
plt.bar(np.linspace(1,n_comp,n_comp),strength_axis)
plt.xlabel('PCA Axis')
plt.ylabel('Overall Strength')
plt.title('Overall Strength of Each PCA Axis')
plt.show()